In [8]:
!pip3 install git+https://github.com/ssut/py-hanspell.git
!pip3 install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-bk050z42
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-bk050z42
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-g6welu50
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-g6welu50
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for argparse>=1.1.0 from https://files.pythonhosted.org/packages/f2/94/3af39d34be01a24a6e65433d19e107099374224905f1e0cc6bbe1fd22a2f/argparse-1.4.0-py2.py3-none-any.whl.metadata
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using c

In [9]:
import pandas as pd
import re
from hanspell import spell_checker
from pykospacing import Spacing

spacing = Spacing()

def preprocess_text(text):
    if not isinstance(text, str): # float type error
        text = str(text)
    
    # 1. HTML 태그 제거
    text = re.sub('<.*?>', '', text)
    # 2. 웃음 표현 통일 (ㅋㅋ, ㅎㅎ)
    text = re.sub(r'[ㅋ]+', 'ㅋㅋ', text)  # 'ㅋ'는 'ㅋㅋ'로 통일
    text = re.sub(r'[ㅎ]+', 'ㅎㅎ', text)  # 'ㅎ'는 'ㅎㅎ'로 통일
    # 3. 특수문자 제거 및 반복된 특수문자 하나로 줄이기
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s.,?!]', '', text)  # 한글, 영문, 숫자, 기본 특수문자만 허용
    text = re.sub(r'([.?!,:;]){2,}', r'\1', text)  # 중복된 특수문자(!!, ??? 등)를 하나로 줄이기
    # 4. 영어 소문자로 변환
    text = text.lower()
    # 5. 문장 시작이 ! 또는 .인 경우 제거
    text = re.sub(r'^[!.]+', '', text)
    # 6. hanspell을 사용한 맞춤법 검사
    try:
        spelled_sent = spell_checker.check(text)
        text = spelled_sent.checked
    except:
        pass  
    # 7. 띄어쓰기 교정 적용
    text = spacing(text)
    # 8. 양 끝 공백 제거
    text = text.strip()

    return text

I0000 00:00:1727318119.212877 1318568 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [10]:
###################################################
input_file = 'data/train.csv'  # 원본 파일 경로
output_file = 'data/train_pre.csv'  # 저장 파일 경로
###################################################

In [ ]:
df = pd.read_csv(input_file)

df['processed_sentence_1'] = df['sentence_1'].apply(preprocess_text)
df['processed_sentence_2'] = df['sentence_2'].apply(preprocess_text)

df.to_csv(output_file, index=False)